In [141]:
%matplotlib inline
from pylab import *
import numpy as np
from collections import deque

In [171]:
class Queue:
    """A queue for use in simulating M/M/1/k.
    
    Attributes:
        k (int): Maximum customers allowed in the system.
        departures (list): A sample of departure intervals.
        queue (list): A deque object.
        dropped (int): Number of items dropped because queue was full.
    """
    
    def __init__(self, k, mu, departures):
        """Forms a queue.

        Args:
            k (int): Maximum customers allowed in the system.
            mu (float): Rate out of the queue.
            departures (int): Number of departure intervals to generate.
        """
        self.k = k
        self.departures = exponential(1/mu, departures)
        self.queue = deque([], k)
        self.dropped = 0
        
    def is_empty(self):
        """Checks if the queue is empty.
        
        Returns:
            True if empty, False otherwise.
        """
        return len(self.queue) is 0
    
    def is_full(self):
        """Checks if the queue is full.
        
        Returns:
            True if full, False otherwise.
        """
        return len(self.queue) is self.k
    
    def enqueue(self, item):
        """Adds an item to end of the queue.
        
        Args:
            item: An item to add to the queue.
        """
        if self.is_full():
            self.dropped += 1
        else:
            self.queue.append(item)
        
    def dequeue(self):
        """Removes the fist item from the queue."""
        if not self.is_empty():
            self.queue.popleft()

In [211]:
def simulation(lamb, mu, k, phi=0.5, samples=6000):
    """Used to run a simulation of an M/M/1/k network.
    
    Args:
        lamb (float): The rate into the entire network.
        mu (float): The rate out of the two queues in the network.
        k (int): Maximum number of customers the two queues can handle. 
        samples (int): Number of packets to sample. Defaults to 6000.
        phi (float): Probability an arrival goes to the first queue.
    """
    queue1 = Queue(k, mu, samples)
    queue2 = Queue(k, mu, samples)
    # A sample of arrivals.
    arrivals = exponential(1/lamb, samples)
    # Counts arrivals to each node.
    queue1_arrivals, queue2_arrivals = 0, 0
    # Count time passed.
    time = 0
    # Indexes for sample space lists.
    j, k, l = 0, 0, 0
    # Iterate over entire sample of arrivals.
    for i in range(0, samples):
        # Idle state, ignores output rates.
        if time is 0:
            if random() < phi:
                queue1_arrivals += 1
                queue1.enqueue("a")
            else:
                queue2_arrivals += 1
                queue2.enqueue("a")
            # Increments time by one arrival interval.
            time += arrivals[i]
        else:
            # Checks if the server processed a packet before the next arrival.
            if queue1.departures[j] <= time or queue2.departures[k] <= time:
                # Dequeues any packets that should have been processed
                # before the next arrival.
                while queue1.departures[j] <= time:
                    queue1.dequeue()
                    # Sums the intervals to compare with time since arrival.
                    queue1.departures[j+1] += queue1.departures[j]
                    j += 1
                while queue2.departures[k] <= time:
                    queue2.dequeue()
                    queue2.departures[k+1] += queue2.departures[k]
                    k += 1
            # Splits arrivals based on phi probability.
            if random() < phi:
                queue1_arrivals += 1
                queue1.enqueue("a")
            else:
                queue2_arrivals += 1
                queue2.enqueue("a")
            # Increments time by one arrival interval.
            time += arrivals[i]
        
    return time, queue1.dropped/queue1_arrivals

def eval_blocking(lamb, mu, k, phi=None):
    if phi is None:
        rho = lamb/mu
    else:
        rho = (lamb*phi)/mu
    return rho**k*((1-rho)/(1-rho**(k+1)))

In [217]:
print(simulation(8, 5, 5, 0.6))

(759.08395692648469, 0.13465291329802062)
0.15013154146406452
